This notebook performs profile normalization.
All profiles are normalized to the DMSO control treated profiles.

In [1]:
import argparse
import os
import pathlib
import sys

import numpy as np
import pandas as pd
from arg_parsing_utils import parse_args
from notebook_init_utils import bandicoot_check, init_notebook
from pycytominer import normalize

root_dir, in_notebook = init_notebook()

profile_base_dir = bandicoot_check(
    pathlib.Path(os.path.expanduser("~/mnt/bandicoot")).resolve(), root_dir
)

In [2]:
if not in_notebook:
    args = parse_args()
    patient = args["patient"]
    image_based_profiles_subparent_name = args["image_based_profiles_subparent_name"]

else:
    patient = "NF0014_T1"
    image_based_profiles_subparent_name = "image_based_profiles"

In [3]:
# pathing
sc_annotated_path = pathlib.Path(
    f"{profile_base_dir}/data/{patient}/{image_based_profiles_subparent_name}/3.qc_profiles/sc_flagged_outliers.parquet"
).resolve(strict=True)
organoid_annotated_path = pathlib.Path(
    f"{profile_base_dir}/data/{patient}/{image_based_profiles_subparent_name}/3.qc_profiles/organoid_flagged_outliers.parquet"
).resolve(strict=True)


# output path
sc_normalized_output_path = pathlib.Path(
    f"{profile_base_dir}/data/{patient}/{image_based_profiles_subparent_name}/4.normalized_profiles/sc_norm.parquet"
).resolve()
organoid_normalized_output_path = pathlib.Path(
    f"{profile_base_dir}/data/{patient}/{image_based_profiles_subparent_name}/4.normalized_profiles/organoid_norm.parquet"
).resolve()

organoid_normalized_output_path.parent.mkdir(parents=True, exist_ok=True)

In [4]:
# read in the data
sc_annotated_profiles = pd.read_parquet(sc_annotated_path)
organoid_annotated_profiles = pd.read_parquet(organoid_annotated_path)

### Normalize the single-cell profiles

In [5]:
sc_metadata_columns = [x for x in sc_annotated_profiles.columns if "Metadata" in x]

sc_metadata_columns += [
    "Area.Size.Shape_Cell_CENTER.X",
    "Area.Size.Shape_Cell_CENTER.Y",
    "Area.Size.Shape_Cell_CENTER.Z",
]
sc_features_columns = [
    col for col in sc_annotated_profiles.columns if col not in sc_metadata_columns
]

In [6]:
# find inf values and replace with NaN
sc_annotated_profiles[sc_features_columns] = sc_annotated_profiles[
    sc_features_columns
].replace([float("inf"), -float("inf")], np.nan)

In [7]:
# normalize the data
sc_normalized_profiles = normalize(
    sc_annotated_profiles,
    features=sc_features_columns,
    meta_features=sc_metadata_columns,
    method="standardize",
    samples="Metadata_treatment == 'DMSO'",
)
sc_normalized_profiles.to_parquet(sc_normalized_output_path, index=False)
sc_normalized_profiles.head()

,Metadata_patient_tumor,Metadata_object_id,Metadata_unit,Metadata_dose,Metadata_treatment,Metadata_image_set,Metadata_Well,Metadata_parent_organoid,Metadata_Target,Metadata_Class,...,Texture_Cytoplasm_Mito_Difference.Entropy_256.3,Texture_Cytoplasm_Mito_Difference.Variance_256.3,Texture_Cytoplasm_Mito_Entropy_256.3,Texture_Cytoplasm_Mito_Information.Measure.of.Correlation.1_256.3,Texture_Cytoplasm_Mito_Information.Measure.of.Correlation.2_256.3,Texture_Cytoplasm_Mito_Inverse.Difference.Moment_256.3,Texture_Cytoplasm_Mito_Sum.Average_256.3,Texture_Cytoplasm_Mito_Sum.Entropy_256.3,Texture_Cytoplasm_Mito_Sum.Variance_256.3,Texture_Cytoplasm_Mito_Variance_256.3
0,NF0014_T1,61,uM,1,Fimepinostat,E5-2,E5,-1,PI3K and HDAC inhibitor,Small Molecule,...,0.064430,0.036350,0.009193,0.541606,-0.013471,0.027300,-0.179517,-0.003906,-0.281164,-0.253400
1,NF0014_T1,114,uM,1,Fimepinostat,E5-2,E5,-1,PI3K and HDAC inhibitor,Small Molecule,...,1.387787,-0.987652,1.137201,0.833670,0.808809,-1.139383,0.929826,1.155166,0.478639,0.580791
2,NF0014_T1,184,uM,1,Fimepinostat,E5-2,E5,-1,PI3K and HDAC inhibitor,Small Molecule,...,1.635019,-1.390974,1.422826,0.874011,0.970971,-1.587359,0.460939,1.453986,-0.415846,-0.386394
3,NF0014_T1,219,uM,1,Fimepinostat,E5-2,E5,-1,PI3K and HDAC inhibitor,Small Molecule,...,-0.071752,0.146210,-0.093037,-0.152675,0.059486,0.159556,-0.587803,-0.106811,-0.802970,-0.831941
4,NF0014_T1,255,uM,1,Fimepinostat,E5-2,E5,-1,PI3K and HDAC inhibitor,Small Molecule,...,0.037290,0.053939,-0.022213,0.395098,-0.011712,0.046180,-0.026482,-0.031981,-0.016887,0.045138


In [8]:
# Replace inf values with NaN for single-cell profiles
sc_annotated_profiles[sc_features_columns] = sc_annotated_profiles[
    sc_features_columns
].replace([np.inf, -np.inf], np.nan)

### Normalize the organoid profiles

In [9]:
organoid_metadata_columns = [
    x for x in organoid_annotated_profiles.columns if "Metadata" in x
]

organoid_metadata_columns += [
    "Area.Size.Shape_Organoid_CENTER.X",
    "Area.Size.Shape_Organoid_CENTER.Y",
    "Area.Size.Shape_Organoid_CENTER.Z",
]
organoid_features_columns = [
    col
    for col in organoid_annotated_profiles.columns
    if col not in organoid_metadata_columns
]
# Replace inf values with NaN for organoid profiles
organoid_annotated_profiles[organoid_features_columns] = organoid_annotated_profiles[
    organoid_features_columns
].replace([np.inf, -np.inf], np.nan)
# normalize the data
organoid_normalized_profiles = normalize(
    organoid_annotated_profiles,
    features=organoid_features_columns,
    meta_features=organoid_metadata_columns,
    method="standardize",
    samples="Metadata_treatment == 'DMSO'",
)
organoid_normalized_profiles.to_parquet(organoid_normalized_output_path, index=False)
organoid_normalized_profiles.head()

,Metadata_patient_tumor,Metadata_object_id,Metadata_unit,Metadata_dose,Metadata_treatment,Metadata_image_set,Metadata_Well,Metadata_single_cell_count,Metadata_Target,Metadata_Class,...,Texture_Organoid_Mito_Difference.Entropy_256.3,Texture_Organoid_Mito_Difference.Variance_256.3,Texture_Organoid_Mito_Entropy_256.3,Texture_Organoid_Mito_Information.Measure.of.Correlation.1_256.3,Texture_Organoid_Mito_Information.Measure.of.Correlation.2_256.3,Texture_Organoid_Mito_Inverse.Difference.Moment_256.3,Texture_Organoid_Mito_Sum.Average_256.3,Texture_Organoid_Mito_Sum.Entropy_256.3,Texture_Organoid_Mito_Sum.Variance_256.3,Texture_Organoid_Mito_Variance_256.3
0,NF0014_T1,2,uM,1,Fimepinostat,E5-2,E5,NaN,PI3K and HDAC inhibitor,Small Molecule,...,-0.799700,0.727493,-0.707564,1.408761,-0.908726,0.723078,-0.593903,-0.733030,-0.528032,-0.561465
1,NF0014_T1,4,uM,1,Fimepinostat,E5-2,E5,NaN,PI3K and HDAC inhibitor,Small Molecule,...,-0.790156,0.722770,-0.701173,-0.081791,-0.791176,0.718448,-0.589805,-0.725724,-0.523691,-0.555467
2,NF0014_T1,7,uM,1,Fimepinostat,E5-2,E5,NaN,PI3K and HDAC inhibitor,Small Molecule,...,-0.799328,0.727338,-0.707337,1.408760,-0.908707,0.722927,-0.593751,-0.732765,-0.527863,-0.561134
3,NF0014_T1,19,uM,1,Fimepinostat,E5-2,E5,NaN,PI3K and HDAC inhibitor,Small Molecule,...,-0.797577,0.726521,-0.706253,1.309233,-0.902352,0.722125,-0.593090,-0.731511,-0.527244,-0.559971
4,NF0014_T1,23,uM,1,Fimepinostat,E5-2,E5,NaN,PI3K and HDAC inhibitor,Small Molecule,...,0.990027,-0.917486,0.847170,-1.158874,1.342547,-0.941170,0.691929,0.914296,0.408973,0.434582
